# Seq trait
## TRAVERSABLE总结
特性（trait) Seq 具有两个子特征（subtrait） LinearSeq和IndexedSeq。它们不添加任何新的操作，但都提供不同的性能特点：线性序列具有高效的 head 和 tail 操作，而索引序列具有高效的apply, length, 和 (如果可变) update操作。

常用线性序列有 scala.collection.immutable.List和scala.collection.immutable.Stream。常用索引序列有 scala.Array scala.collection.mutable.ArrayBuffer。Vector 类提供一个在索引访问和线性访问之间有趣的折中。它同时具有高效的恒定时间的索引开销，和恒定时间的线性访问开销。正因为如此，对于混合访问模式，vector是一个很好的基础。后面将详细介绍vector。

Buffers是可变序列一个重要的种类。它们不仅允许更新现有的元素，而且允许元素的插入、移除和在buffer尾部高效地添加新元素。buffer 支持的主要新方法有：用于在尾部添加元素的 += 和 ++=；用于在前方添加元素的+=: 和 ++=: ；用于插入元素的 insert和insertAll；以及用于删除元素的 remove 和 -=。如下表所示。

ListBuffer和ArrayBuffer是常用的buffer实现 。顾名思义，ListBuffer依赖列表（List），支持高效地将它的元素转换成列表。而ArrayBuffer依赖数组（Array），能快速地转换成数组。

### 序列的操作分类
序列的操作有以下几种，如下表所示：
<ul>
  <li><strong>索引和长度的操作</strong> apply、isDefinedAt、length、indices，及lengthCompare。序列的apply操作用于索引访问；因此，Seq[T]类型的序列也是一个以单个Int（索引下标）为参数、返回值类型为T的偏函数。换言之，Seq[T]继承自Partial Function[Int, T]。序列各元素的索引下标从0开始计数，最大索引下标为序列长度减一。序列的length方法是collection的size方法的别名。lengthCompare方法可以比较两个序列的长度，即便其中一个序列长度无限也可以处理。</li>
  <li><strong>索引检索操作</strong>（indexOf、lastIndexOf、indexofSlice、lastIndexOfSlice、indexWhere、lastIndexWhere、segmentLength、prefixLength）用于返回等于给定值或满足某个谓词的元素的索引。</li>
  <li><strong>加法运算</strong>（+:，:+，padTo）用于在序列的前面或者后面添加一个元素并作为新序列返回。</li>
  <li><strong>更新操作</strong>（updated，patch）用于替换原序列的某些元素并作为一个新序列返回。</li>
  <li><strong>排序操作</strong>（sorted, sortWith, sortBy）根据不同的条件对序列元素进行排序。</li>
  <li><strong>反转操作</strong>（reverse, reverseIterator, reverseMap）用于将序列中的元素以相反的顺序排列。</li>
  <li><strong>比较</strong>（startsWith, endsWith, contains, containsSlice, corresponds）用于对两个序列进行比较，或者在序列中查找某个元素。</li>
  <li><strong>多集操作</strong>（intersect, diff, union, distinct）用于对两个序列中的元素进行类似集合的操作，或者删除重复元素。</li>
</ul>

如果一个序列是可变的，它提供了另一种更新序列中的元素的，但有副作用的update方法，Scala中常有这样的语法，如seq(idx) = elem。它只seq.update(idx, elem)的简写，所以update 提供了方便的赋值语法。应注意update 和updated之间的差异。update 再原来基础上更改序列中的元素，并且仅适用于可变序列。而updated 适用于所有的序列，它总是返回一个新序列，而不会修改原序列。

### Seq类的操作
| WHAT IT IS              | WHAT IT DOES                                                                                                                                    |
| ----------------------- | ----------------------------------------------------------------------------------------------------------------------------------------------- |
| **索引和长度**          |                                                                                                                                                 |
| xs(i)                   | (或者写作xs apply i)。xs的第i个元素                                                                                                             |
| xs isDefinedAt i        | 测试xs.indices中是否包含i。                                                                                                                     |
| xs.length               | 序列的长度（同size）。                                                                                                                          |
| xs.lengthCompare ys     | 如果xs的长度小于ys的长度，则返回-1。如果xs的长度大于ys的长度，则返回+1，如果它们长度相等，则返回0。即使其中一个序列是无限的，也可以使用此方法。 |
| xs.indices              | xs的索引范围，从0到xs.length - 1。                                                                                                              |
| **索引搜索**            |                                                                                                                                                 |
| xs indexOf x            | 返回序列xs中等于x的第一个元素的索引（存在多种变体）。                                                                                           |
| xs lastIndexOf x        | 返回序列xs中等于x的最后一个元素的索引（存在多种变体）。                                                                                         |
| xs indexOfSlice ys      | 查找子序列ys，返回xs中匹配的第一个索引。                                                                                                        |
| xs indexOfSlice ys      | 查找子序列ys，返回xs中匹配的倒数一个索引。                                                                                                      |
| xs indexWhere p         | xs序列中满足p的第一个元素。（有多种形式）                                                                                                       |
| xs segmentLength (p, i) | xs中，从xs(i)开始并满足条件p的元素的最长连续片段的长度。                                                                                        |
| xs prefixLength p       | xs序列中满足p条件的先头元素的最大个数。                                                                                                         |
| **加法**：              |                                                                                                                                                 |
| x +: xs                 | 由序列xs的前方添加x所得的新序列。                                                                                                               |
| xs :+ x                 | 由序列xs的后方追加x所得的新序列。                                                                                                               |
| xs padTo (len, x)       | 在xs后方追加x，直到长度达到len后得到的序列。                                                                                                    |
| **更新**                |                                                                                                                                                 |
| xs patch (i, ys, r)     | 将xs中第i个元素开始的r个元素，替换为ys所得的序列。                                                                                              |
| xs updated (i, x)       | 将xs中第i个元素替换为x后所得的xs的副本。                                                                                                        |
| xs(i) = x               | （或写作 xs.update(i, x)，仅适用于可变序列）将xs序列中第i个元素修改为x。                                                                        |
| **排序**                |                                                                                                                                                 |
| xs.sorted               | 通过使用xs中元素类型的标准顺序，将xs元素进行排序后得到的新序列。                                                                                |
| xs sortWith lt          | 将lt作为比较操作，并以此将xs中的元素进行排序后得到的新序列。                                                                                    |
| xs sortBy f             | 将序列xs的元素进行排序后得到的新序列。参与比较的两个元素各自经f函数映射后得到一个结果，通过比较它们的结果来进行排序。                           |
| **反转**                |                                                                                                                                                 |
| xs.reverse              | 与xs序列元素顺序相反的一个新序列。                                                                                                              |
| xs.reverseIterator      | 产生序列xs中元素的反序迭代器。                                                                                                                  |
| xs reverseMap f         | 以xs的相反顺序，通过f映射xs序列中的元素得到的新序列。                                                                                           |
| **比较**                |                                                                                                                                                 |
| xs startsWith ys        | 测试序列xs是否以序列ys开头（存在多种形式）。                                                                                                    |
| xs endsWith ys          | 测试序列xs是否以序列ys结束（存在多种形式）。                                                                                                    |
| xs contains x           | 测试xs序列中是否存在一个与x相等的元素。                                                                                                         |
| xs containsSlice ys     | 测试xs序列中是否存在一个与ys相同的连续子序列。                                                                                                  |
| (xs corresponds ys)(p)  | 测试序列xs与序列ys中对应的元素是否满足二元的判断式p。                                                                                           |
| **多集操作**            |                                                                                                                                                 |
| xs intersect ys         | 序列xs和ys的交集，并保留序列xs中的顺序。                                                                                                        |
| xs diff ys              | 序列xs和ys的差集，并保留序列xs中的顺序。                                                                                                        |
| xs union ys             | 并集；同xs ++ ys。                                                                                                                              |
| xs.distinct             | 不含重复元素的xs的子序列。                                                                                                                      |


### Buffer类的操作
| WHAT IT IS            | WHAT IT DOES                                   |
| --------------------- | ---------------------------------------------- |
| **加法**：            |                                                |
| buf += x              | 将元素x追加到buffer，并将buf自身作为结果返回。 |
| buf += (x, y, z)      | 将给定的元素追加到buffer。                     |
| buf ++= xs            | 将xs中的所有元素追加到buffer。                 |
| x +=: buf             | 将元素x添加到buffer的前方。                    |
| xs ++=: buf           | 将xs中的所有元素都添加到buffer的前方。         |
| buf insert (i, x)     | 将元素x插入到buffer中索引为i的位置。           |
| buf insertAll (i, xs) | 将xs的所有元素都插入到buffer中索引为i的位置。  |
| **移除**：            |                                                |
| buf -= x              | 将元素x从buffer中移除。                        |
| buf remove i          | 将buffer中索引为i的元素移除。                  |
| buf remove (i, n)     | 将buffer中从索引i开始的n个元素移除。           |
| buf trimStart n       | 移除buffer中的前n个元素。                      |
| buf trimEnd n         | 移除buffer中的后n个元素。                      |
| buf.clear()           | 移除buffer中的所有元素。                       |
| **克隆**：            |                                                |
| buf.clone             | 与buf具有相同元素的新buffer。                  |

## seq操作测试
### 索引和长度的操作 
apply、isDefinedAt、length、indices，及lengthCompare。序列的apply操作用于索引访问；因此，Seq[T]类型的序列也是一个以单个Int（索引下标）为参数、返回值类型为T的偏函数。换言之，Seq[T]继承自Partial Function[Int, T]。序列各元素的索引下标从0开始计数，最大索引下标为序列长度减一。序列的length方法是collection的size方法的别名。lengthCompare方法可以比较两个序列的长度，即便其中一个序列长度无限也可以处理。

#### xs(i)	
(或者写作xs apply i)。xs的第i个元素

In [1]:
val list = List(1,2,3,4,5,6)

list: List[Int] = List(1, 2, 3, 4, 5, 6)

In [2]:
list(1)

res1: Int = 2

In [3]:
list(0)

res2: Int = 1

In [4]:
//java.lang.IndexOutOfBoundsException: 6
//list(6)

//语法错误
//list[1]

#### xs isDefinedAt i	
测试xs.indices中是否包含i。

In [5]:
list.isDefinedAt _

res4: Int => Boolean = <function1>

In [6]:
list.isDefinedAt(0)

res5: Boolean = true

In [7]:
list.isDefinedAt(6)

res6: Boolean = false

#### xs.length	序列的长度（同size）

In [8]:
list.length

res7: Int = 6

#### xs.lengthCompare ys	
如果xs的长度小于ys的长度，则返回-1。如果xs的长度大于ys的长度，则返回+1，如果它们长度相等，则返回0。即使其中一个序列是无限的，也可以使用此方法。

In [11]:
//编译错误
//list.lengthCompare(List(1,2,3))
list.lengthCompare _

res10: Int => Int = <function1>

In [12]:
list.lengthCompare(1)

res11: Int = 1

In [13]:
list.lengthCompare(6)

res12: Int = 0

In [14]:
list.lengthCompare(7)

res13: Int = -1

#### xs.indices	
xs的索引范围，从0到xs.length - 1。

In [15]:
list.indices

res14: Range = Range(0, 1, 2, 3, 4, 5)

### 索引检索操作
（indexOf、lastIndexOf、indexofSlice、lastIndexOfSlice、indexWhere、lastIndexWhere、segmentLength、prefixLength）用于返回等于给定值或满足某个谓词的元素的索引。

  def indexOf[B >: A](elem: B, from: Int): Int

  def indexOf[B >: A](elem: B): Int = indexOf(elem, 0)

In [17]:
list.indexOf(3)

res16: Int = 2

In [18]:
list.indexOf(3,2)

res17: Int = 2

In [19]:
list.indexOf(3,3)

res18: Int = -1

### 加法运算（+:，:+，padTo）
用于在序列的前面或者后面添加一个元素并作为新序列返回。

x +: xs	由序列xs的前方添加x所得的新序列。

xs :+ x	由序列xs的后方追加x所得的新序列。

xs padTo (len, x)	在xs后方追加x，直到长度达到len后得到的序列。

def padTo[B >: A, That](len: Int, elem: B)(implicit bf: CanBuildFrom[Repr, B, That]): That

In [20]:
list :+ 9

res19: List[Int] = List(1, 2, 3, 4, 5, 6, 9)

In [21]:
9 +: list

res20: List[Int] = List(9, 1, 2, 3, 4, 5, 6)

In [22]:
list.padTo(10,9)

res21: List[Int] = List(1, 2, 3, 4, 5, 6, 9, 9, 9, 9)

### 更新操作（updated，patch）
用于替换原序列的某些元素并作为一个新序列返回。

xs patch (i, ys, r)	将xs中第i个元素开始的r个元素，替换为ys所得的序列。

xs updated (i, x)	将xs中第i个元素替换为x后所得的xs的副本。

xs(i) = x	（或写作 xs.update(i, x)，仅适用于可变序列）将xs序列中第i个元素修改为x。


In [23]:
list.updated(0,8)

res22: List[Int] = List(8, 2, 3, 4, 5, 6)

In [24]:
list(0)

res23: Int = 1

In [25]:
val listBuffer = scala.collection.mutable.ListBuffer(1,2,3,4,5)
listBuffer(0) = 10

listBuffer: collection.mutable.ListBuffer[Int] = ListBuffer(10, 2, 3, 4, 5)

In [26]:
list.patch(1,listBuffer,2)

res25: List[Int] = List(1, 10, 2, 3, 4, 5, 4, 5, 6)

In [27]:
list.patch(1,listBuffer,5)

res26: List[Int] = List(1, 10, 2, 3, 4, 5)

In [28]:
list.patch(1,listBuffer,0)

res27: List[Int] = List(1, 10, 2, 3, 4, 5, 2, 3, 4, 5, 6)

In [30]:
list.patch(0,listBuffer,0)

res29: List[Int] = List(10, 2, 3, 4, 5, 1, 2, 3, 4, 5, 6)

### 排序操作（sorted, sortWith, sortBy）
根据不同的条件对序列元素进行排序。
#### xs.sorted	
通过使用xs中元素类型的标准顺序，将xs元素进行排序后得到的新序列。

  def sorted[B >: A](implicit ord: Ordering[B]): Repr

In [32]:
List(3,4,1,2,6).sorted

res31: List[Int] = List(1, 2, 3, 4, 6)

#### xs sortWith lt	
将lt作为比较操作，并以此将xs中的元素进行排序后得到的新序列。

  def sortWith(lt: (A, A) => Boolean): Repr

In [36]:
List(3,4,1,2,6).sortWith(_.compareTo(_) < 0)

res35: List[Int] = List(1, 2, 3, 4, 6)

In [37]:
List(3,4,1,2,6).sortWith(_.compareTo(_) > 0)

res36: List[Int] = List(6, 4, 3, 2, 1)

#### xs sortBy f	
将序列xs的元素进行排序后得到的新序列。参与比较的两个元素各自经f函数映射后得到一个结果，通过比较它们的结果来进行排序。类似python中的排序函数  

  def sortBy[B](f: A => B)(implicit ord: Ordering[B]): Repr = sorted(ord on f)

In [40]:
List("1","4","2","6","5").sortBy(_.toInt)

res39: List[String] = List("1", "2", "4", "5", "6")

### 反转操作（reverse, reverseIterator, reverseMap）
用于将序列中的元素以相反的顺序排列。

In [41]:
list.reverse

res40: List[Int] = List(6, 5, 4, 3, 2, 1)

### 多集操作（intersect, diff, union, distinct）
用于对两个序列中的元素进行类似集合的操作，或者删除重复元素。

xs intersect ys	序列xs和ys的交集，并保留序列xs中的顺序。  
xs diff ys	序列xs和ys的差集，并保留序列xs中的顺序。  
xs union ys	并集；同xs ++ ys。  
xs.distinct	不含重复元素的xs的子序列。  

In [44]:
val left = List(1,2,3,4,5,6,6)

left: List[Int] = List(1, 2, 3, 4, 5, 6, 6)

In [45]:
val right = List(4,4,5,6,7,8,9)

right: List[Int] = List(4, 4, 5, 6, 7, 8, 9)

In [47]:
left.distinct

res46: List[Int] = List(1, 2, 3, 4, 5, 6)

In [48]:
right.distinct

res47: List[Int] = List(4, 5, 6, 7, 8, 9)

In [49]:
left.union(right)

res48: List[Int] = List(1, 2, 3, 4, 5, 6, 6, 4, 4, 5, 6, 7, 8, 9)

In [50]:
left ++ right

res49: List[Int] = List(1, 2, 3, 4, 5, 6, 6, 4, 4, 5, 6, 7, 8, 9)

In [52]:
left.intersect(right)

res51: List[Int] = List(4, 5, 6)

In [53]:
left.diff(right)

res52: List[Int] = List(1, 2, 3, 6)

In [54]:
left.diff(List(1))

res53: List[Int] = List(2, 3, 4, 5, 6, 6)

### 可变序列的索引
#### 加法
buf += x	将元素x追加到buffer，并将buf自身作为结果返回。  
buf += (x, y, z)	将给定的元素追加到buffer。  
buf ++= xs	将xs中的所有元素追加到buffer。  
x +=: buf	将元素x添加到buffer的前方。  
xs ++=: buf	将xs中的所有元素都添加到buffer的前方。  
buf insert (i, x)	将元素x插入到buffer中索引为i的位置。  
buf insertAll (i, xs)	将xs的所有元素都插入到buffer中索引为i的位置。  

In [55]:
listBuffer

res54: collection.mutable.ListBuffer[Int] = ListBuffer(10, 2, 3, 4, 5)

In [56]:
listBuffer += 6

res55: collection.mutable.ListBuffer[Int] = ListBuffer(10, 2, 3, 4, 5, 6)

In [57]:
listBuffer += (7,8,9)

res56: collection.mutable.ListBuffer[Int] = ListBuffer(10, 2, 3, 4, 5, 6, 7, 8, 9)

In [58]:
listBuffer ++= list

res57: collection.mutable.ListBuffer[Int] = ListBuffer(10, 2, 3, 4, 5, 6, 7, 8, 9, 1, 2, 3, 4, 5, 6)

In [59]:
listBuffer --= list

res58: collection.mutable.ListBuffer[Int] = ListBuffer(10, 7, 8, 9, 2, 3, 4, 5, 6)

In [60]:
1 +=: listBuffer

res59: collection.mutable.ListBuffer[Int] = ListBuffer(1, 10, 7, 8, 9, 2, 3, 4, 5, 6)

In [61]:
List(2,3) ++=: listBuffer

res60: collection.mutable.ListBuffer[Int] = ListBuffer(2, 3, 1, 10, 7, 8, 9, 2, 3, 4, 5, 6)

In [63]:
listBuffer.insert(1,9)

In [64]:
listBuffer

res63: collection.mutable.ListBuffer[Int] = ListBuffer(2, 9, 3, 1, 10, 7, 8, 9, 2, 3, 4, 5, 6)

In [65]:
listBuffer.insert(1,9,9,9)

In [66]:
listBuffer

res65: collection.mutable.ListBuffer[Int] = ListBuffer(2, 9, 9, 9, 9, 3, 1, 10, 7, 8, 9, 2, 3, 4, 5, 6)

In [67]:
listBuffer.insertAll(0,list)

In [68]:
listBuffer

res67: collection.mutable.ListBuffer[Int] = ListBuffer(1, 2, 3, 4, 5, 6, 2, 9, 9, 9, 9, 3, 1, 10, 7, 8, 9, 2, 3, 4, 5, 6)

#### 移除
buf -= x	将元素x从buffer中移除。  
buf remove i	将buffer中索引为i的元素移除。  
buf remove (i, n)	将buffer中从索引i开始的n个元素移除。  
buf trimStart n	移除buffer中的前n个元素。  
buf trimEnd n	移除buffer中的后n个元素。  
buf.clear()	移除buffer中的所有元素。  

In [69]:
listBuffer.clear()

In [70]:
listBuffer ++= list

res69: collection.mutable.ListBuffer[Int] = ListBuffer(1, 2, 3, 4, 5, 6)

In [71]:
listBuffer -= 2

res70: collection.mutable.ListBuffer[Int] = ListBuffer(1, 3, 4, 5, 6)

In [72]:
listBuffer.remove(1)

res71: Int = 3

In [73]:
listBuffer

res72: collection.mutable.ListBuffer[Int] = ListBuffer(1, 4, 5, 6)

In [74]:
listBuffer.remove(1,2)

In [75]:
listBuffer

res74: collection.mutable.ListBuffer[Int] = ListBuffer(1, 6)

In [76]:
listBuffer ++= list

res75: collection.mutable.ListBuffer[Int] = ListBuffer(1, 6, 1, 2, 3, 4, 5, 6)

In [77]:
listBuffer.trimStart(3)

In [78]:
listBuffer

res77: collection.mutable.ListBuffer[Int] = ListBuffer(2, 3, 4, 5, 6)

In [79]:
listBuffer.trimEnd(3)

In [80]:
listBuffer

res79: collection.mutable.ListBuffer[Int] = ListBuffer(2, 3)

In [81]:
listBuffer.trimEnd(3)
listBuffer

res80_1: collection.mutable.ListBuffer[Int] = ListBuffer()